In [1]:
!pip install sagemaker --upgrade -q

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip install --upgrade pip' command.


In [ ]:
#!aws s3 cp val s3://dc-ds-ali-playground/Dog-Cat/val --recursive

In [ ]:
#!aws s3 cp train s3://dc-ds-ali-playground/Dog-Cat/train --recursive

In [2]:
# SageMaker setup
import sagemaker

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket='dc-ds-ali-playground')
bucket = sess.default_bucket()

In [13]:
from sagemaker.pytorch import PyTorch

pytorch_estimator = PyTorch('pytorch-train.py',
                            instance_type='ml.m5.large',
                            instance_count=1,
                            framework_version='1.8.0',
                            py_version='py3',
                            role=role,
                            hyperparameters = {'epochs': 1, 'batch-size': 4, 'learning-rate': 0.01})

In [14]:
pytorch_estimator.fit({'train': 's3://dc-ds-ali-playground/Dog-Cat/train',
                        'val': 's3://dc-ds-ali-playground/Dog-Cat/val'})

2021-08-13 09:37:29 Starting - Starting the training job...
2021-08-13 09:37:51 Starting - Launching requested ML instancesProfilerReport-1628847448: InProgress
......
2021-08-13 09:38:51 Starting - Preparing the instances for training......
2021-08-13 09:39:58 Downloading - Downloading input data.....................
2021-08-13 09:43:18 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-08-13 09:43:18,944 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-08-13 09:43:18,946 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-08-13 09:43:18,957 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-08-13 09:43:21,978 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-08-13 09:43:22,367 sagemaker-training-to